In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf 
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

In [ ]:
data = pd.read_csv("C:\\Users\\wajit\\Documents\\GitHub\\Project-ANN\\2017-2022.csv", parse_dates=['Tanggal'], date_parser=lambda x: pd.to_datetime(x, format='%d/%m/%Y'))

In [ ]:
print(data)

In [ ]:
data.describe()

In [ ]:
data['Tanggal'] = pd.to_datetime(data['Tanggal'], format='%H:%M:%S')
latitude = data["Latitude"]
longitude = data["Longitude"]
kedalaman = data["Kedalaman (km)"]
magnitude = data["Magnitude"]

In [ ]:
checkpoint = ModelCheckpoint('C:\\Users\\wajit\\Documents\\GitHub\\Project-ANN\\checkpoints\\model_1.h5', monitor='val_mae', verbose=1, save_best_only=True)

In [ ]:
gempa_semua = []
for index in range(len(latitude)):
  gempa = []
  gempa.append(latitude[index])
  gempa.append(longitude[index])
  gempa.append(kedalaman[index])

  gempa_semua.append(gempa)

In [ ]:
gempa_semua = np.array(gempa_semua)

In [ ]:
print(gempa_semua)

In [ ]:
magnitude = magnitude.astype('float64')

In [ ]:
magnitude.dtype

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(gempa_semua, magnitude, test_size=0.2, random_state=42)

In [ ]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(3,),  kernel_initializer='random_uniform'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(48, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='linear'))
model.summary()

In [ ]:
optimizer = Adam(learning_rate=0.001)

In [ ]:
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

In [ ]:
history = model.fit(X_train, y_train, batch_size=64, epochs=3000, verbose=1, validation_data=(X_test, y_test), callbacks=[checkpoint])

In [ ]:
model = load_model("C:\\Users\\wajit\\Documents\\GitHub\\Project-ANN\\checkpoints\\model_1.h5")
error = model.evaluate(X_test, y_test)[1]
error_percentage =error * 100
print("Loss: ", error_percentage)

In [ ]:
latitude_subset = latitude[:100]
longitude_subset = longitude[:100]
kedalaman_subset = kedalaman[:100]

In [ ]:
count = 0  # Variable to count the number of predicted data
predictions = []  # List to store the predicted values

for i in range(len(latitude)):
    for j in range(len(longitude)):
        for k in range(len(kedalaman)):
            koordinat_latitude = float(latitude[i])
            koordinat_longitude = float(longitude[j])
            kedalaman_gempa = float(kedalaman[k])

            input_data = np.array([[koordinat_latitude, koordinat_longitude, kedalaman_gempa]])
            hasil_prediksi = model.predict(input_data)

            # Append the predicted value to the list
            predictions.append(hasil_prediksi)

            # Increase the count by 1
            count += 1

            # Print the prediction
            print("Perkiraan kekuatan gempa {} Mg".format(hasil_prediksi))

            if count == 6149:
                break
        else:
            continue
        break
    else:
        continue
    break

# Convert the list of predictions to a NumPy array
predictions = np.array(predictions)

print("Banyaknya data yang diprediksi:", count)


In [ ]:
#koordinat_latitude = float(input("Masukkan koordinat latitude anda : "))
#koordinat_longitude = float(input("Masukkan koordinat longitude anda: "))
#kedalaman_gempa = float(input("Masukkan Kedalaman Gempa dalam (Km): "))

In [ ]:
#prediksi_gempa = model.predict([[koordinat_latitude, koordinat_longitude, kedalaman_gempa]])
#print("Perkiraan kekuatan gempa {} Mg".format(prediksi_gempa))

In [ ]:
print(predictions)

In [ ]:
flattened_array = predictions.flatten()

In [ ]:
magnitude_subset = magnitude[:100]

In [ ]:
x = data['Tanggal']
y_data_asli = magnitude
y_data_forecasted = flattened_array

In [ ]:
plt.plot(x, y_data_asli, label='Data Asli')
plt.plot(x, y_data_forecasted, label='Hasil Pelatihan')
plt.xlabel('Time')
plt.ylabel('magnitude')
plt.title('Data Asli vs Hasil Pelatihan')
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['mae'])
plt.plot(history.history['val_mae'])
plt.xlabel('epochs')
plt.ylabel('error')
plt.title('Perbandingan')
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())
plt.legend(['Original', 'Forecast'], loc='upper right')
plt.show()